In [1]:
#importing data from xml file
import xml.etree.ElementTree as ET
x_train_tree = ET.parse('training.xml')
x_train_root=x_train_tree.getroot()

In [2]:
#for child in x_train_root:
#    print(child.tag, child.attrib, child.text)

In [3]:
#priting first sentence from train data for checking
print(x_train_root[0].text)


		hesitate in to giving is @aapleaks #aapsweep revenge should this statehood take way bjp not the #aapstorm best 
	


In [4]:
#importing train lables
y_train_tree=ET.parse('AnnotationTraining.xml')
y_train_root=y_train_tree.getroot()

In [6]:
#checking train label by printing first line
print(y_train_root[0].text)


		en en en en en X X en en en en en en NE en en X en 
	


In [8]:
Indian_lang=['mr','hi','te','ta','kn','gu','bn','ml'] #for labeling sentences
Y=['mr','hi','te','ta','kn','gu','bn','ml'] 
req_labels=Indian_lang

In [10]:
#formatting the train data
train_data_X = {}
train_data_Y = {}
pos=0
for i in range(len(x_train_root)):
    word=x_train_root[i].text.split()
    label=y_train_root[i].text.split()
    if len(word)!=len(label): #error lines skipped
        print("Excluded error line : ", i+1, " ", len(word), " ", len(label));
    else:
        temp=list()
        train_data_Y[pos]='en'
        for j in range(len(label)):
            if label[j] in req_labels:
                temp.append(word[j])
            if label[j] in Y:
                train_data_Y[pos]=label[j]
        train_data_X[pos]=temp
        pos+=1
keys = list(set(train_data_Y[dic] for dic in train_data_Y)) 
print("\nUnique Y Labels: \n\n",str(keys))

Excluded error line :  320   24   25
Excluded error line :  429   30   31
Excluded error line :  503   183   184
Excluded error line :  546   22   23
Excluded error line :  1201   57   58
Excluded error line :  1465   25   27
Excluded error line :  1788   47   48
Excluded error line :  1987   12   13
Excluded error line :  2083   22   24
Excluded error line :  2172   55   56
Excluded error line :  2372   21   22
Excluded error line :  2407   31   32
Excluded error line :  2488   50   51

Unique Y Labels: 

 ['mr', 'en', 'kn', 'te', 'ta', 'bn', 'gu', 'ml', 'hi']


In [11]:
#checking
print("Words: ", train_data_X[1], "\n\nLabel: ", train_data_Y[1])

Words:  ['Tinbeku', 'anstide'] 

Label:  kn


In [12]:
#prep model 
#sentence level n-grams
def getNGram(wordlist, n):
    ngram = []
    for i in range(len(wordlist)-(n-1)):
        ngram.append(wordlist[i:i+n])
    return ngram

In [13]:
#sentence level for making low to high grams
def getNGrams(wordlist, low ,high):
    ngrams_list = []
    for i in range(low,high+1):
        ngrams_list.extend(getNGram(wordlist,i))
    return ngrams_list

In [14]:
#dataset level low to high grams
def getNGrams(wordlist, low ,high):
    ngrams_list = []
    for word in wordlist:
        for i in range(low,high+1): 
            ngrams_list.extend(getNGram(list(word),i))
    for i in range(low,high+1):
        ngrams_list.extend(getNGram(wordlist,i))
    return ngrams_list

In [15]:
#checking
print(getNGrams(train_data_X[2],1,3))

[['N'], ['a'], ['m'], ['e'], ["'"], ['l'], ['a'], ['N', 'a'], ['a', 'm'], ['m', 'e'], ['e', "'"], ["'", 'l'], ['l', 'a'], ['N', 'a', 'm'], ['a', 'm', 'e'], ['m', 'e', "'"], ['e', "'", 'l'], ["'", 'l', 'a'], ['o'], ['r'], ['u'], ['o', 'r'], ['r', 'u'], ['o', 'r', 'u'], ['o'], ['n'], ['n'], ['u'], ['o', 'n'], ['n', 'n'], ['n', 'u'], ['o', 'n', 'n'], ['n', 'n', 'u'], ['p'], ['a'], ['n'], ['n'], ['e'], ['n'], ['p', 'a'], ['a', 'n'], ['n', 'n'], ['n', 'e'], ['e', 'n'], ['p', 'a', 'n'], ['a', 'n', 'n'], ['n', 'n', 'e'], ['n', 'e', 'n'], ["Name'la"], ['oru'], ['onnu'], ['pannen'], ["Name'la", 'oru'], ['oru', 'onnu'], ['onnu', 'pannen'], ["Name'la", 'oru', 'onnu'], ['oru', 'onnu', 'pannen']]


In [16]:
#testing/checking
print(['s', 't', 'i'] in getNGrams(train_data_X[1],1,3))

True


In [20]:
#lang wise => ngrams/features
Lang_Fea={}
low=1
high=4
for i in train_data_Y:
    if train_data_Y[i] not in Lang_Fea:
        Lang_Fea[train_data_Y[i]]=[]
    Lang_Fea[train_data_Y[i]].extend(getNGrams(train_data_X[i],low,high))

In [21]:
#ngrams for 'kn' lang
print(Lang_Fea['kn'])

[['T'], ['i'], ['n'], ['b'], ['e'], ['k'], ['u'], ['T', 'i'], ['i', 'n'], ['n', 'b'], ['b', 'e'], ['e', 'k'], ['k', 'u'], ['T', 'i', 'n'], ['i', 'n', 'b'], ['n', 'b', 'e'], ['b', 'e', 'k'], ['e', 'k', 'u'], ['T', 'i', 'n', 'b'], ['i', 'n', 'b', 'e'], ['n', 'b', 'e', 'k'], ['b', 'e', 'k', 'u'], ['a'], ['n'], ['s'], ['t'], ['i'], ['d'], ['e'], ['a', 'n'], ['n', 's'], ['s', 't'], ['t', 'i'], ['i', 'd'], ['d', 'e'], ['a', 'n', 's'], ['n', 's', 't'], ['s', 't', 'i'], ['t', 'i', 'd'], ['i', 'd', 'e'], ['a', 'n', 's', 't'], ['n', 's', 't', 'i'], ['s', 't', 'i', 'd'], ['t', 'i', 'd', 'e'], ['Tinbeku'], ['anstide'], ['Tinbeku', 'anstide'], ['S'], ['u'], ['r'], ['y'], ['a'], ['S', 'u'], ['u', 'r'], ['r', 'y'], ['y', 'a'], ['S', 'u', 'r'], ['u', 'r', 'y'], ['r', 'y', 'a'], ['S', 'u', 'r', 'y'], ['u', 'r', 'y', 'a'], ['b'], ['a'], ['n'], ['d'], ['a'], ['a'], ['g'], ['a'], ['b', 'a'], ['a', 'n'], ['n', 'd'], ['d', 'a'], ['a', 'a'], ['a', 'g'], ['g', 'a'], ['b', 'a', 'n'], ['a', 'n', 'd'], ['n', 'd'

In [22]:
#set of all langs/labels in train dataset
keys = list(set(dic for dic in Lang_Fea)) 
print("\nUnique Y Labels in Lang_Fea: \n\n",str(keys))


Unique Y Labels in Lang_Fea: 

 ['mr', 'en', 'kn', 'te', 'ta', 'bn', 'gu', 'ml', 'hi']


In [24]:
#testing data X and Y fetch
x_test_tree = ET.parse('testing.xml')
x_test_root=x_test_tree.getroot()
y_test_tree=ET.parse('AnnotationTesting.xml')
y_test_root=y_test_tree.getroot()

In [26]:
#processing the test data according to required format
test_data_X = {}
test_data_Y = {} #sentence label
test_data_Y_word_labels={} #word labels in sentence
pos=0
for i in range(len(x_test_root)):
    word=x_test_root[i].text.split()
    label=y_test_root[i].text.split()
    if len(word)!=len(label):
        print("Excluded error line : ", i+1, " ", len(word), " ", len(label));
    else:
        temp=list()
        test_data_Y_word_labels[pos]=list()
        test_data_Y[pos]='en'
        for j in range(len(label)):
            if label[j] in req_labels:
                temp.append(word[j])
                test_data_Y[pos]=label[j]
                test_data_Y_word_labels[pos].append(label[j])
        test_data_X[pos]=temp
        pos+=1
langs = list(set(test_data_Y[dic] for dic in test_data_Y)) 
print("\nUnique Y Labels: \n\n",str(langs))

Excluded error line :  380   130   131
Excluded error line :  391   45   47
Excluded error line :  471   23   24
Excluded error line :  538   34   35
Excluded error line :  660   23   24
Excluded error line :  724   7   8
Excluded error line :  728   22   23

Unique Y Labels: 

 ['mr', 'en', 'kn', 'te', 'ta', 'bn', 'gu', 'ml', 'hi']


In [27]:
#prediction section
import numpy as np

sentence_predictions={}

def predict_1(wordlist,label,index):
    ngrams_wordlist=getNGrams(wordlist,low,high)
    Predict_Y={}
    for gram in ngrams_wordlist:
        lang_lvl_gram_count={}
        net_lang_gram_count=0
        for lang in langs:
            if gram in Lang_Fea[lang]:
                net_lang_gram_count+=1
                if lang not in lang_lvl_gram_count:
                    lang_lvl_gram_count[lang]=1
                else:
                    lang_lvl_gram_count[lang]+=1
        if net_lang_gram_count!=0:
            for item in lang_lvl_gram_count:
                count_item=lang_lvl_gram_count[item]
                if count_item!=0:
                    if item not in Predict_Y:
                        Predict_Y[item]=0
                    Predict_Y[item]+=(count_item/net_lang_gram_count)*len(gram)
       
    prediction=['en'] #pure english sentence
    best=0
    for item in Predict_Y:
        if Predict_Y[item] >= best:
            prediction.clear()
            prediction.append(item)
            best=Predict_Y[item]
    sentence_predictions[index]=prediction
    if 'en' in prediction: #doesn't belong to any of the given 8 indian langs...only english
        return True
    #print(Predict_Y,"\n\n")
    #print(prediction, label)
    return label in prediction 

In [28]:
#testing for a random sentence
predict_1(test_data_X[5],test_data_Y[5],0)

True

In [29]:
#to predict all
def predict_all():
    correct=0
    total=0
    for item in test_data_X:
        if predict_1(test_data_X[item],test_data_Y[item],total)==True:
            correct+=1
        total+=1
    return correct/total

In [30]:
accuracy=predict_all()
print("sentence level classifier accuraccy: ",accuracy)

sentence level classifier accuraccy:  0.8280254777070064


In [31]:
#LAYER 2 (WORD LEVEL CLASSIFIER)
#after classsifing the sentence with label 'x' -> we then use "'x' vs 'en'" binary classifier to classify at word level

In [33]:
#Binary classifiers i.e., "'x' vs 'en'" classifiers for x in Indian_lang given above
Lang_Words={} #to hold the lang wise ngrams for modeling binary classifiers 
File_bengali=open('Classifier/bengaliW.txt','r')
Lang_Words['bn']=File_bengali.read().split()

File_gujarati=open('Classifier/gujaratiW.txt','r')
Lang_Words['gu']=File_gujarati.read().split()

File_hindi=open('Classifier/hindiW.txt','r')
Lang_Words['hi']=File_hindi.read().split()

File_kannada=open('Classifier/kannadaW.txt','r')
Lang_Words['kn']=File_kannada.read().split()

File_malayalam=open('Classifier/malayalamW.txt','r')
Lang_Words['ml']=File_malayalam.read().split()

File_marati=open('Classifier/maratiW.txt','r')
Lang_Words['mr']=File_marati.read().split()

File_tamil=open('Classifier/tamil.txt','r')
Lang_Words['ta']=File_tamil.read().split()

File_telugu=open('Classifier/telugu.txt','r')
Lang_Words['te']=File_telugu.read().split()

In [35]:
#no of words for each language
for item in Lang_Words:
    print(item, " ", len(Lang_Words[item]))

bn   4342
gu   2753
hi   3267
kn   9406
ml   3693
mr   1048576
ta   1248
te   7867


In [36]:
#from above output we see 'ma' to have too many words..
#'ma' is bottleneck... solution => limit max entries to 10000..
print('mr', " ", len(Lang_Words['mr'][0:10000]))

mr   10000


In [37]:
#lang low to high value grams maker ..i.e., creating features for each language
def lang_maker(wordlist,low,high):
    temp=getNGrams(wordlist,low,high)
    Lang_list = []
    #[Lang_list.append(x) for x in temp if x not in Lang_list]
    [Lang_list.append(x) for x in temp] #replacememt for above; promote duplicate entries 
    return Lang_list

In [39]:
#to hold lang wise grams/features
Lang_ngrams={}
#function to generate ngrams for all langs ie., modeling 8 binary classifiers ( one for each lang)
def gen():
    for lang in Lang_Words:
        Lang_ngrams[lang]=[]
        if lang is 'mr':
            Lang_ngrams[lang].extend(lang_maker(Lang_Words[lang][0:10000],1,5))
        else:
            Lang_ngrams[lang].extend(lang_maker(Lang_Words[lang],1,5))

In [40]:
#fetching 'en' lang grams (already prepared and kept in a file)
File_english=open('Classifier/eng2.txt','r')
temp=File_english.read().split()
Lang_ngrams['en']=[]
#[Lang_ngrams['en'].append(list(x)) for x in temp if list(x) not in Lang_ngrams['en']]
[Lang_ngrams['en'].append(list(x)) for x in temp] #replacememt for above; promote duplicate entries 
#print(Lang_ngrams['en'])
print(" ")

In [41]:
#calling function to create ngrams for each lang
gen()

In [42]:
#printing 'en' features/grams 
Lang_ngrams['en']

[['t'],
 ['h'],
 ['e'],
 ['t', 'h'],
 ['h', 'e'],
 ['t', 'h', 'e'],
 ['a'],
 ['n'],
 ['d'],
 ['a', 'n'],
 ['n', 'd'],
 ['a', 'n', 'd'],
 ['o'],
 ['f'],
 ['o', 'f'],
 ['t'],
 ['o'],
 ['t', 'o'],
 ['a'],
 ['i'],
 ['n'],
 ['i', 'n'],
 ['i'],
 ['s'],
 ['i', 's'],
 ['f'],
 ['o'],
 ['r'],
 ['f', 'o'],
 ['o', 'r'],
 ['f', 'o', 'r'],
 ['o'],
 ['n'],
 ['o', 'n'],
 ['t'],
 ['h'],
 ['a'],
 ['t'],
 ['t', 'h'],
 ['h', 'a'],
 ['a', 't'],
 ['t', 'h', 'a'],
 ['h', 'a', 't'],
 ['t', 'h', 'a', 't'],
 ['w'],
 ['i'],
 ['t'],
 ['h'],
 ['w', 'i'],
 ['i', 't'],
 ['t', 'h'],
 ['w', 'i', 't'],
 ['i', 't', 'h'],
 ['w', 'i', 't', 'h'],
 ['b'],
 ['e'],
 ['b', 'e'],
 ['a'],
 ['r'],
 ['e'],
 ['a', 'r'],
 ['r', 'e'],
 ['a', 'r', 'e'],
 ['y'],
 ['o'],
 ['u'],
 ['y', 'o'],
 ['o', 'u'],
 ['y', 'o', 'u'],
 ['a'],
 ['s'],
 ['a', 's'],
 ['w'],
 ['a'],
 ['s'],
 ['w', 'a'],
 ['a', 's'],
 ['w', 'a', 's'],
 ['s'],
 ['b'],
 ['y'],
 ['b', 'y'],
 ['a'],
 ['t'],
 ['a', 't'],
 ['i'],
 ['t'],
 ['i', 't'],
 ['i'],
 ['f'],
 ['r'],
 [

In [43]:
#binanry prediction section
import numpy as np

def predict_binary_word(word, vsLang,label):
    ngrams_word=getNGrams(list(word),1,5)
    Predict_Y={}
    langs=['en']
    langs.append(vsLang)
    for gram in ngrams_word:
        lang_lvl_gram_count={}
        net_lang_gram_count=0
        for lang in langs:
            if gram in Lang_ngrams[lang]:
                net_lang_gram_count+=1
                #net_lang_gram_count+=Lang_ngrams[lang].count(gram)
                if lang not in lang_lvl_gram_count:
                    lang_lvl_gram_count[lang]=1
                    #lang_lvl_gram_count[lang]=Lang_ngrams[lang].count(gram) #replacement
                else:
                    lang_lvl_gram_count[lang]+=1
                    #lang_lvl_gram_count[lang]+=Lang_ngrams[lang].count(gram)
        if net_lang_gram_count!=0:
            for item in lang_lvl_gram_count:
                count_item=lang_lvl_gram_count[item]
                if count_item!=0:
                    if item not in Predict_Y:
                        Predict_Y[item]=0
                    Predict_Y[item]+=(count_item/net_lang_gram_count)*len(gram)
        #print(lang_lvl_gram_count)
        #print(net_lang_gram_count)
       
    prediction='en' #default prediction
    best=0
    for item in Predict_Y:
        if Predict_Y[item] >= best:
            prediction=item
            best=Predict_Y[item]
    
    if 'en' == prediction: #doesn't belong to any of the given 8 langs
        return True
    
    #print(ngrams_word)
    #print(Predict_Y,"\n\n")
    #print(prediction, label)
    return label in prediction 

In [44]:
#checking
print(predict_binary_word('baby','hi','en'))
#ngrams_word=getNGrams(list('there'),1,5)
#print(ngrams_word)

True


In [46]:
#printing 'hi' lang words
print(Lang_Words['hi'])

['sau', 'aansoo', 'roye', 'do', 'akhiyan', 'sau', 'maara', 'hai', 'jinhe', 'neendon', 'ne', 'kabhi', 'na', 'soye', 'wo', 'dil', 'ka', 'musaafir', 'baat', 'maane', 'mud', 'dekhe', 'raste', 'puraane', 'zidd', 'chhodta', 'hi', 'nahi', 'dekh', 'raha', 'kaanch', 'ke', 'sapne', 'ik', 'din', 'hain', 'jo', 'tidakne', 'ishq', 'mein', 'tooti', 'cheeze', 'jag', 'koi', 'jodta', 'jaane', 'aasoon', 'akhiyaan', 'ankhiyan', 'o', 'waala', 'se', 'uske', 'jaan', 'gayi', 'aate', 'jaate', 'phir', 'dikhe', 'palkon', 'chhaane', 'mod', 'kayi', 'guzra', 'huaa', 'pal', 'uska', 'laga', 'rang', 'utra', 'hothon', 'chhuti', 'gaye', 'saajan', 'ki', 'batiyan', 'aansu', 'kaise', 'roke', 'bhalaa', 'haath', 'laqeerein', 'chhod', 'gayin', 'laakhon', 'tukde', 'kiye', 'dhadkan', 'ko', 'bhi', 'tod', 'theharega', 'dard', 'kahin', 'gunjaaish', 'dikhti', 'chanda', 'bin', 'jiya', 'lage', 'jal', 'kaajal', 'si', 'ratiyan', 'jinhein', 'woh', 'fizool', 'tere', 'mile', 'mujhko', 'zindagi', 'mujhe', 'faasle', 'naa', 'dena', 'tu', 'ab

In [47]:
#checing number of features/grams for each lang
for item in Lang_ngrams:
    print(item, " no of ngrams ", len(Lang_ngrams[item]))

en  no of ngrams  122234
bn  no of ngrams  101704
gu  no of ngrams  63659
hi  no of ngrams  68934
kn  no of ngrams  263561
ml  no of ngrams  138107
mr  no of ngrams  280456
ta  no of ngrams  38862
te  no of ngrams  481480


In [49]:
#testing train data accurracy (finding train loss) for a lgiven lang
def train_accuracy(lang):
  positive=0.0
  total=0.0
  for item in Lang_Words[lang][0:100]:
    if predict_binary_word(item,lang,lang)==True:
      positive+=1.0
    total+=1.0
  print(lang ," accuraccy on train:  ", positive, " out of ", total)

In [50]:
#finding train loss for all given langs (by testing for first 100 words)
for item in Lang_ngrams:
  if item != "en":
    train_accuracy(item)

bn  accuraccy on train:   100.0  out of  100.0
gu  accuraccy on train:   100.0  out of  100.0
hi  accuraccy on train:   100.0  out of  100.0
kn  accuraccy on train:   100.0  out of  100.0
ml  accuraccy on train:   100.0  out of  100.0
mr  accuraccy on train:   100.0  out of  100.0
ta  accuraccy on train:   100.0  out of  100.0
te  accuraccy on train:   100.0  out of  100.0


In [51]:
#integration of above two layers (sentence lvl classifier followed by word level)
def test_accuracy_sentence(sentence,lang_predicted,true_word_labels):
    positive=0
    count=0
    for word in sentence:
        #print("\n",word , " ", true_word_labels[count])
        if predict_binary_word(word,lang_predicted,true_word_labels[count])==True:
            positive+=1
            #print("bingo")
        count+=1
    #print(positive, " out of ", count, " correctly predicted")
    return [float(positive),float(count)]    

In [52]:
#checking for a random sentence
test_accuracy_sentence('ohh yeah boy acchi'.split(),'hi',['en','en','en','hi'])

[3.0, 4.0]

In [55]:
#checking data format.. as it needs to be send as an argument for test_accuracy-sentence function
print(test_data_X[78])
print(test_data_Y_word_labels[78])
print(test_data_Y[78])
print(sentence_predictions[78][0])
#predict_1(test_data_X[78],test_data_Y[78],78)
#print(test_accuracy_sentence(test_data_X[4],test_data_Y_word_labels[4],sentence_predictions[4][0]))

['brajesh', 'kumar', 'phir', 'bhi', 'namo']
['hi', 'hi', 'hi', 'hi', 'hi']
hi
hi


In [56]:
#lang wise correct predcition
lang_wise_pos={}
#totol prediction lang wise
lang_wise_tot={}    
#predict word labels for entire test data
def test_all():
    pos=0
    tot=0
    for item in test_data_X:
        res=list()
        #print(item)
        if len(test_data_X[item])!=0:
            #print("in ",item)
            res=test_accuracy_sentence(test_data_X[item],sentence_predictions[item][0],test_data_Y_word_labels[item])
            if len(res)==2:
                if test_data_Y[item] not in lang_wise_pos:
                    lang_wise_pos[test_data_Y[item]]=res[0]
                else:
                    lang_wise_pos[test_data_Y[item]]+=res[0]
                if test_data_Y[item] not in lang_wise_tot:
                    lang_wise_tot[test_data_Y[item]]=res[1]
                else:
                    lang_wise_tot[test_data_Y[item]]+=res[1]    
                pos+=res[0]
                tot+=res[1]
                #print(float(pos)/float(tot))
    return [pos,tot]

In [57]:
#result lang wise
result=test_all()
for item in lang_wise_tot:
    if item in lang_wise_pos:
        print (item, " accuracy: ", float(float(lang_wise_pos[item]/lang_wise_tot[item])))
    else:
        print (item, " accuracy: 0")
print('\n\n Final overall accuracy is (by combining both the layers) ', float(float(result[0])/float(result[1])))

kn  accuracy:  0.8277591973244147
te  accuracy:  0.9325626204238922
bn  accuracy:  0.9773226042428675
hi  accuracy:  0.8745294855708908
mr  accuracy:  0.9757709251101322
ml  accuracy:  0.8040201005025126
ta  accuracy:  1.0
gu  accuracy:  0.15675675675675677


 Final overall accuracy is (by combining both the layers)  0.8946693533614215
